In [0]:
#default_exp deployment

# Server Implementation

Let's now take what we had before and run inference based on a list of filenames. We'll make a quick script to get the ball rolling for how we want everything to do using `nbdev` again

In [0]:
#hide
#Run once per session
!pip install fastai2

In [0]:
#hide
from nbdev.showdoc import *

We'll want the libraries we've used

In [0]:
#export
from fastai2.vision.all import *
from fastai2.basics import *

Including our new `style_transfer.py` file

In [0]:
#export
from style_transfer import *

Let's grab our original style image

In [0]:
learn = load_learner('myModel', cpu=False)

And now we can make and prepare our dataloader with a filename!

In [0]:
dset = Datasets('cat.jpg', tfms=[PILImage.create])
dl = dset.dataloaders(after_item=[ToTensor()], after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)], bs=1)

In [0]:
t_im = dl.one_batch()[0]

And get our raw output. 

In [0]:
with torch.no_grad():
  res = learn.model(t_im)

Let's wrap this into a function

In [0]:
#export
def get_learner(fn, cpu=False):
  return load_learner(fn, cpu=cpu)

In [0]:
#export
def make_datasets(learn, fns, bs=1):
  cuda = next(learn.model.parameters()).is_cuda
  dset = Datasets(fns, tfms=[PILImage.create])
  if cuda: 
    after_batch = [IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)] 
    dl = dset.dataloaders(after_item=[ToTensor()], after_batch=after_batch, bs=1)
  else: 
    after_batch = [Normalize.from_stats(*imagenet_stats, cuda=False)]
    dl = dset.dataloaders(after_item=[ToTensor()], after_batch=after_batch, bs=1, device='cpu')
  return dl

In [0]:
#export
from torchvision.utils import save_image

We can write a quick `save_im` function to save all our outputed tensors to images

In [0]:
#export
def save_im(imgs:list, path):
  "Save a n*c*w*h `Tensor` into seperate images"
  [save_image(im, f'{path}/{i}.png') for i, im in enumerate(imgs)]

Now let's put it all together

In [0]:
#export
def inference(pkl_name, fnames:list, path:Path, cpu:bool=True):
  "Grab inference on a model, filenames, and a path to save it to"
  path = path/'results'
  path.mkdir(parents=True, exist_ok=True)
  learn = get_learner(pkl_name, cpu)
  if len(fnames) > 1:
    dls = []
    for fname in fnames:
      dls.append(make_datasets(learn, fnames, 1))
  else:
    dls = [make_datasets(learn, fnames, 1)]
  res = []
  for b in dls:
    t_im = b.one_batch()[0]
    with torch.no_grad():
      out = learn.model(t_im)
    res.append(out)
  save_im(res, path)

And try it out!

In [0]:
fnames = ['cat.jpg'] * 5

In [0]:
inference('myModel', fnames, path=Path(''))

Lastly let's make a `.py` file again to run it off of

In [0]:
#hide
from nbdev.imports import *
from nbdev.export import reset_nbdev_module, notebook2script

create_config('myLib', user='muellerzr', path='.', cfg_name='settings.ini')
cfg = Config(cfg_name='settings.ini')

In [0]:
#hide
reset_nbdev_module()

In [0]:
#hide
from nbdev.export import notebook2script
notebook2script('05_Inference_Server.ipynb')

Converted 05_Inference_Server.ipynb.


And we're done!